In [1]:
import numpy as np
from time import time
from driver import Model

In [2]:
myip = Model("bitstream/cnn.bit")

In [22]:
myip.debug=False

In [4]:
# load test data, check accuracy against software implementation
data = np.int32(np.load("test_x.npy"))
gold = np.load("test_y.npy")

In [5]:
myip.setCNNWeights(np.load("CNN_weights.npy"))
myip.setCNNBias(np.load("CNN_bias.npy"))
myip.setDenseWeights(np.load("dense_weights.npy"))
myip.setDenseBias(np.load("dense_bias.npy"))

In [21]:
# sample data
print(data[0,0,:])
# dataset size
print(data.shape[0])

test_size = 10
test_size = data.shape[0]

[ -5427  -4580   2832  -1489  -6022 -11662]
736


In [25]:
myip.set_threshold(0.98)
inference_count = 0
threshld_miss_count = 0
confusion_matrix = np.zeros((4,4))

start_time = time()

player0_current_data_index = 0
player0_current_inference_step = 0
player0_predicted = 0
player0_done = False
player0_gold_result = np.argmax(gold[player0_current_data_index])
player0_prediction_above_threshold = False

player1_current_data_index = test_size//2
player1_current_inference_step = 0
player1_predicted = 0
player1_done = False
player1_gold_result = np.argmax(gold[player1_current_data_index])
player1_prediction_above_threshold = False

while not player0_done or not player1_done:
    # player 1 part
    if not player0_done:
        # do inference
#         print(f"player 1 inferencing at {player0_current_data_index}, {player0_current_inference_step}")
#         print(f"reference gold result: {player0_gold_result}")
        player0_predicted = myip.inference(data[player0_current_data_index,\
                                                player0_current_inference_step,\
                                                :], \
                                           user_number=0)
        inference_count += 1
        if player0_predicted != -1:
            player0_prediction_above_threshold = True
            player0_gold_result = np.argmax(gold[player0_current_data_index])
            confusion_matrix[player0_gold_result,player0_predicted] += 1
            player0_current_inference_step = data.shape[1]
            
        # prep for next inference
        player0_current_inference_step += 1
        if player0_current_inference_step >= data.shape[1]:
            if not player0_prediction_above_threshold:
                threshld_miss_count += 1
            player0_current_data_index += 1
            player0_current_inference_step = 0
            player0_prediction_above_threshold = False
            myip.resetBuffer(user_number=0)
#             print("player0 reset")
        if player0_current_data_index == test_size//2:
            print("player0 done")
            player0_done = True
    
    # player 2 part
    if not player1_done:
        # do inference
#         print(f"player 2 inferencing at {player1_current_data_index - data.shape[0]//2}, {player1_current_inference_step}")
#         print(f"reference gold result: {player1_gold_result}")
        player1_predicted = myip.inference(data[player1_current_data_index,\
                                                player1_current_inference_step,\
                                                :], \
                                           user_number=1)
        inference_count += 1
        if player1_predicted != -1:
            player1_prediction_above_threshold = True
            player1_gold_result = np.argmax(gold[player1_current_data_index])
            confusion_matrix[player1_gold_result,player1_predicted] += 1
            player1_current_inference_step = data.shape[1]
            
        # prep for next inference
        player1_current_inference_step += 1
        if player1_current_inference_step >= data.shape[1]:
            if not player1_prediction_above_threshold:
                threshld_miss_count += 1
            player1_current_data_index += 1
            player1_current_inference_step = 0
            player1_prediction_above_threshold = False
            myip.resetBuffer(user_number=1)
#             print("player1 reset")
        if player1_current_data_index == test_size:
            print("player1 done")
            player1_done = True


# for i in range(0, data.shape[0]):
    
# #     print("-"*40 + f"Data: {i}" + "-"*40)
#     gold_result = np.argmax(gold[i])
#     prediction_above_threshold = False
    
#     for j in range(0, data.shape[1]):
        
#         # run inference
#         predicted = myip.inference(data[i,j,:])
#         inference_count += 1
#         if predicted != -1:
#             prediction_above_threshold = True
#             break
        
#     if prediction_above_threshold:
#         confusion_matrix[gold_result,predicted] += 1
#     else:
#         print("not confident enough to give prediction")
#         threshld_miss_count += 1
#     myip.resetBuffer()

print(f"total time for {inference_count} inferences = {time() - start_time}")
print(f"Average inference time = {(time() - start_time)/inference_count}")
print(f"Max inference frequency = {inference_count/(time() - start_time)}")
print(f"Miss rate = {threshld_miss_count/test_size*100}%")

print(confusion_matrix)

player1 done
player0 done
total time for 12693 inferences = 10.599830865859985
Average inference time = 0.000835125547562481
Max inference frequency = 1197.3879078358832
Miss rate = 0.8152173913043478%
[[162.   0.   0.   2.]
 [  1. 134.   3.   1.]
 [  0.  10. 226.   4.]
 [  0.   0.   1. 186.]]
